In [1]:
from datetime import datetime, timedelta
import cx_Oracle
import warnings
warnings.filterwarnings("ignore")
cx_Oracle.init_oracle_client(lib_dir=r"/Users/kaka/Downloads/instantclient_18_1")
import pandas as pd
import calendar

from datetime import datetime, timedelta



def mysql_info():
    """数据上传"""
    username = 'nppbuf'
    password = 'Svwnppbuf321'
    hostname = '10.122.6.59'
    port = 1521
    service_name = 'b2bt'
    dsn = cx_Oracle.makedsn(hostname, port, service_name=service_name)
    # 连接 Oracle 数据库
    connection = cx_Oracle.connect(user=username, password=password, dsn=dsn)
    return connection

In [2]:
def get_data_from_oracle(connection):

    sql_query = """  
                 SELECT     
    * from nppbuf.T_DD_DEMAND_hist_fj
    """
    # 执行 SQL 查询，并按批次抽取数据
    chunksize = 100000  # 每次查询返回的行数，你可以根据需要调整这个值
    query_chunks = pd.read_sql(sql_query, connection, chunksize=chunksize)
    # 遍历查询结果的每个批次，并将其合并为一个 DataFrame
    data = pd.DataFrame()
    for chunk in query_chunks:
        data = data.append(chunk)
    # 关闭数据库连接
    # 输出合并后的 DataFrame
    return data

def get_pred_data_from_oracle(connection):

    sql_query = """  
                 SELECT     
    * from nppbuf.T_DD_FORECAST_DETAIL_test
    """
    # 执行 SQL 查询，并按批次抽取数据
    chunksize = 100000  # 每次查询返回的行数，你可以根据需要调整这个值
    query_chunks = pd.read_sql(sql_query, connection, chunksize=chunksize)
    # 遍历查询结果的每个批次，并将其合并为一个 DataFrame
    data = pd.DataFrame()
    for chunk in query_chunks:
        data = data.append(chunk)
    # 关闭数据库连接
    connection.close()
    # 输出合并后的 DataFrame
    return data



connection = mysql_info()
real_demods_info = get_data_from_oracle(connection)
print('真实条目数',real_demods_info.shape[0])
pred_demods_info = get_pred_data_from_oracle(connection)
print('预测条目数',pred_demods_info.shape[0])

真实条目数 7498276
预测条目数 4326366


In [3]:
real_demods_info['YEAR'] = [int(x) for x in real_demods_info['YEAR']]
real_demods_info['MONTH'] = [int(x) for x in real_demods_info['MONTH']]
real_demods_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7498276 entries, 0 to 98275
Data columns (total 6 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   HOSTPARTID           object
 1   HOSTLOCID            object
 2   DSHOSTID             object
 3   YEAR                 int64 
 4   MONTH                int64 
 5   TOTAL_HISTORYAMOUNT  int64 
dtypes: int64(3), object(3)
memory usage: 400.5+ MB


In [43]:
comparis_predic_info = pd.merge(pred_demods_info,real_demods_info,on = 
                                ['HOSTPARTID','HOSTLOCID','DSHOSTID','YEAR','MONTH'],how ='left')

comparis_predic_info.fillna(0,inplace =True)
comparis_predic_info = comparis_predic_info[(comparis_predic_info['MONTH']>=1)&(comparis_predic_info['YEAR']==2023)]
comparis_predic_info

,HOSTPARTID,HOSTLOCID,DSHOSTID,YEAR,MONTH,PRED_VALUES,TOTAL_HISTORYAMOUNT
0,1KD615301D,1000-1,MANUAL,2023,5,4,0.0
1,1KD615301D,3001,DEALER,2023,5,297,285.0
2,1KD615301D,3002,DEALER,2023,5,17,24.0
3,1KD615301D,5001,DEALER,2023,5,148,193.0
4,1KD615301D,5001,MANUAL,2023,5,0,12.0
...,...,...,...,...,...,...,...
4326361,06B100035C,1000-1,DEALER,2023,9,0,0.0
4326362,06F133833B,1000-1,DEALER,2023,9,0,0.0
4326363,0BH325025J Z0H#V,1000-3,DEALER,2023,9,0,0.0
4326364,113857117B,1000-1,DEALER,2023,9,0,0.0


In [44]:
comparis_predic_info['TOTAL_HISTORYAMOUNT']= [1.111 if x ==0 else x for x in 
                                              comparis_predic_info['TOTAL_HISTORYAMOUNT']]

comparis_predic_info['mape'] = (abs(comparis_predic_info['PRED_VALUES'] - comparis_predic_info['TOTAL_HISTORYAMOUNT']) 
                                 / comparis_predic_info['TOTAL_HISTORYAMOUNT']) 
comparis_predic_info.head(20)


,HOSTPARTID,HOSTLOCID,DSHOSTID,YEAR,MONTH,PRED_VALUES,TOTAL_HISTORYAMOUNT,mape
0,1KD615301D,1000-1,MANUAL,2023,5,4,1.111,2.600360
1,1KD615301D,3001,DEALER,2023,5,297,285.000,0.042105
2,1KD615301D,3002,DEALER,2023,5,17,24.000,0.291667
3,1KD615301D,5001,DEALER,2023,5,148,193.000,0.233161
4,1KD615301D,5001,MANUAL,2023,5,0,12.000,1.000000
5,1KD615301D,5002,DEALER,2023,5,159,126.000,0.261905
6,1KD615301D,6000,DEALER,2023,5,306,402.000,0.238806
7,1KD615301D,8011,DEALER,2023,5,285,298.000,0.043624
8,1KD615301D,8012,DEALER,2023,5,257,333.000,0.228228
9,1KD615301D,8013,DEALER,2023,5,183,251.000,0.270916


In [45]:
comparis_predic_info.sort_values(by=['mape'])

,HOSTPARTID,HOSTLOCID,DSHOSTID,YEAR,MONTH,PRED_VALUES,TOTAL_HISTORYAMOUNT,mape
3457655,04E115561S,5002,DEALER,2023,3,2775,2775.000,0.000000
846438,18D853717A 9B9,5001,DEALER,2023,1,6,6.000,0.000000
1814305,1ZD919064,1000-1,DEALER,2023,1,10,10.000,0.000000
3084710,18A821105,8023,DEALER,2023,2,8,8.000,0.000000
846435,18D845202A,8023,DEALER,2023,1,9,9.000,0.000000
...,...,...,...,...,...,...,...,...
1094676,GCN052167Z4,8023,DEALER,2023,9,55254,1.111,49732.573357
1643082,04E115561C,8023,DEALER,2023,10,62749,1.111,56478.747975
1622338,04E115561C,8023,DEALER,2023,9,63226,1.111,56908.090909
2633587,06J115403K,1000-1,DEALER,2023,10,83623,1.111,75267.226823


In [49]:
comparis_predic_info[(comparis_predic_info['HOSTPARTID']=='GVW052195M2#V')
                     &(comparis_predic_info['HOSTLOCID']=='1000-3')
                &(comparis_predic_info['DSHOSTID']=='DEALER')].sort_values(by=['YEAR','MONTH'])


,HOSTPARTID,HOSTLOCID,DSHOSTID,YEAR,MONTH,PRED_VALUES,TOTAL_HISTORYAMOUNT,mape
4310751,GVW052195M2#V,1000-3,DEALER,2023,1,0,1.111,1.000000
4311538,GVW052195M2#V,1000-3,DEALER,2023,2,0,1.111,1.000000
4311607,GVW052195M2#V,1000-3,DEALER,2023,3,0,1.111,1.000000
4312573,GVW052195M2#V,1000-3,DEALER,2023,4,0,1.111,1.000000
4321000,GVW052195M2#V,1000-3,DEALER,2023,5,34,1.111,29.603060
4321069,GVW052195M2#V,1000-3,DEALER,2023,6,55,1.111,48.504950
4321138,GVW052195M2#V,1000-3,DEALER,2023,7,67,1.111,59.306031
4322101,GVW052195M2#V,1000-3,DEALER,2023,8,72,1.111,63.806481
4322170,GVW052195M2#V,1000-3,DEALER,2023,9,72,1.111,63.806481
4323133,GVW052195M2#V,1000-3,DEALER,2023,10,66,1.111,58.405941


In [50]:
real_demods_info[(real_demods_info['HOSTPARTID']=='GVW052195M2#V')
                     &(real_demods_info['HOSTLOCID']=='1000-3')
                &(real_demods_info['DSHOSTID']=='DEALER')].sort_values(by=['YEAR','MONTH'])


,HOSTPARTID,HOSTLOCID,DSHOSTID,YEAR,MONTH,TOTAL_HISTORYAMOUNT
40193,GVW052195M2#V,1000-3,DEALER,2018,1,324
93685,GVW052195M2#V,1000-3,DEALER,2018,2,276
29434,GVW052195M2#V,1000-3,DEALER,2018,3,348
52680,GVW052195M2#V,1000-3,DEALER,2018,4,384
64451,GVW052195M2#V,1000-3,DEALER,2018,5,192
74332,GVW052195M2#V,1000-3,DEALER,2018,6,264
91531,GVW052195M2#V,1000-3,DEALER,2022,1,324
51074,GVW052195M2#V,1000-3,DEALER,2022,2,276
99657,GVW052195M2#V,1000-3,DEALER,2022,3,348
73334,GVW052195M2#V,1000-3,DEALER,2022,4,384


In [48]:
comparis_predic_info[comparis_predic_info['mape']>2]

,HOSTPARTID,HOSTLOCID,DSHOSTID,YEAR,MONTH,PRED_VALUES,TOTAL_HISTORYAMOUNT,mape
0,1KD615301D,1000-1,MANUAL,2023,5,4,1.111,2.600360
11,1KD615301D,8021,DEALER,2023,5,15,2.000,6.500000
26,1KD820015F,8021,DEALER,2023,5,4,1.000,3.000000
45,1KD915333A,1000-1,MANUAL,2023,5,15,1.111,12.501350
112,1ZD807049,3002,DEALER,2023,5,7,2.000,2.500000
...,...,...,...,...,...,...,...,...
4322189,ZSA051002,1000-3,DEALER,2023,9,32,1.111,27.802880
4323133,GVW052195M2#V,1000-3,DEALER,2023,10,66,1.111,58.405941
4323150,Z00970004,1000-3,DEALER,2023,10,29,1.111,25.102610
4323151,ZSA043005,1000-3,DEALER,2023,10,99,1.111,88.108911


In [25]:
# 计算mape，wmape 占比情况

# 定义区间
import matplotlib.pyplot as plt
import seaborn as sns

bins = [0, 0.10, 0.30, 0.50, 1.00, 100000]

# 使用cut函数将数据分成不同区间
comparis_predic_info['ScoreRange'] = pd.cut(comparis_predic_info['mape'], bins=bins, labels=['0-10%', '10%-30%', '30%-50%', '50%-100%', '>100%'],right =False)

comparis_predic_info['ScoreRange'].replace('NaN','0',inplace =True)
# 计算每个区间中的数据数量

grouped = comparis_predic_info.groupby(['MONTH','ScoreRange']).size().reset_index(name='sku_mm_values')
# 计算每个月份内的总样本数
monthly_counts = comparis_predic_info.groupby(['MONTH']).size().reset_index(name='mm_values')

# 合并统计结果
result = pd.merge(grouped, monthly_counts, on=['MONTH'],how = 'left')

# 计算占比
result['Proportion'] = result['sku_mm_values'] / result['mm_values']
result = result[['MONTH','ScoreRange','Proportion']]
result


,MONTH,ScoreRange,Proportion
0,5,0-10%,0.120485
1,5,10%-30%,0.166890
2,5,30%-50%,0.125129
3,5,50%-100%,0.183519
4,5,>100%,0.403977
5,6,0-10%,0.086018
6,6,10%-30%,0.152093
7,6,30%-50%,0.114687
8,6,50%-100%,0.138662
9,6,>100%,0.508540


In [11]:
# from datetime import datetime, timedelta


# def last_days_of_month_and_six_months_ago(date):
#     # 获取当前月份的最后一天
#     last_day_current_month = date.replace(day=1, month=date.month + 1)

#     # 计算6个月前的日期
#     six_months_ago = date - timedelta(days=180)

#     # 获取6个月前月份的最后一天
#     last_day_six_months_ago = six_months_ago.replace(day=1, month=six_months_ago.month + 1)

#     return last_day_current_month.strftime("%Y-%m-%d"), last_day_six_months_ago.strftime("%Y-%m-%d")

# current_date = datetime.now()
# now_time_future, end_time_train = \
#             last_days_of_month_and_six_months_ago(current_date)

In [12]:
test_sql = """  
                 SELECT     
    * from nppbuf.T_DD_FORECAST_DETAIL_FJHIS  where HOSTPARTID='06K109210L' and HOSTLOCID ='1000-1' 
    and DSHOSTID = 'DEALER'
    """
    # 执行 SQL 查询，并按批次抽取数据
connection = mysql_info()
    
query_chunks = pd.read_sql(test_sql, connection) 

In [13]:
query_chunks["year"] = pd.to_datetime(query_chunks['HISTORYBEGDATE']).dt.year.astype(int)
query_chunks["month"] = pd.to_datetime(query_chunks['HISTORYBEGDATE']).dt.month.astype(int)
real_sum_counts = query_chunks.groupby(['HOSTPARTID','HOSTLOCID','DSHOSTID',
                                        'year','month']).sum().reset_index()
real_sum_counts.to_excel('异常需求记录.xlsx')